# Reference
- [llama2+langchain](https://zhuanlan.zhihu.com/p/652172969)
- [RAG QA](https://python.langchain.com/docs/use_cases/question_answering/)
- [langchain vector embedding database](https://ithelp.ithome.com.tw/articles/10327164)
- [LangChain: LLM RAG & Agents](https://zhuanlan.zhihu.com/p/654662274)
- [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/)
- [dspy](https://github.com/stanfordnlp/dspy)
- [LangChain Installation](https://ithelp.ithome.com.tw/articles/10318758)


- [Pipe package github](https://github.com/JulienPalard/Pipe)
- [numexpr package github](https://github.com/pydata/numexpr)

- [llama2+langchain chat](https://blog.futuresmart.ai/integrating-llama-2-with-hugging-face-and-langchain)
- [chroma vector db api](https://docs.trychroma.com/api-reference)
- [chroma vector db embeddings](https://docs.trychroma.com/embeddings)


# PAPER

- [StoryQA](https://knowledge-nlp.github.io/aaai2023/papers/017-StoryQA-poster.pdf)
- [DSPy](https://arxiv.org/pdf/2310.03714.pdf)


# Question
- from langchain.agents import load_tools ?
- wikipedia ?



In [1]:
# !pip install huggingface_hub tensorboard langchain sentence_transformers
# !pip install wikipedia
# !pip install numexpr
# !pip install sentence_transformers

# # !pip install langchain
# !pip install huggingface_hub
# !pip install wikipedia
# !pip install numexpr
# !pip install sentence_transformers

# !pip install openai==0.28.1
# !pip install Chromadb
# !pip install -U openai


# !pip install InstructorEmbedding
# !pip install sentence_transformers
# !pip install Chromadb

# !pip install accelerate
# !pip install bitsandbytes
# !pip install llama-cpp-python


# # !pip install openai==0.28.1
# !pip install langchain openai chromadb langchainhub



# !pip uninstall openai
# !pip install typing-extensions
# !pip install protobuf
# !pip install --upgrade protobuf
# !pip install transformers sentence_transformers

In [72]:
import os
from langchain.llms import OpenAI


import os
from langchain.llms import OpenAI
import inspect
import wikipedia
import re

import copy



In [3]:
import os
with open('../../../../secrets/openai_chat_secret.txt', 'r') as f:
    # openai.api_key = f.read()
    os.environ["OPENAI_API_KEY"] = f.read()

with open('../../../../secrets/huggingface_hub_secret.txt', 'r') as f:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = f.read()


In [4]:

llm = OpenAI(temperature=0.9)  # model_name="text-davinci-003"
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Colorful Toe Treads.


In [5]:
# Load documents

from langchain.document_loaders import WebBaseLoader

# web_loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

# Document Loader
from langchain.document_loaders import TextLoader
text_loader = TextLoader('./books/The Dandelion Girl.txt')
# documents = loader.load()


# PDF Loader
# pdf_file_path = "./finance.pdf"
# from langchain.document_loaders import PyPDFLoader

# pdf_loader = PyPDFLoader(pdf_file_path)

In [6]:
# Split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
loader = text_loader
splitted_docs = text_splitter.split_documents(loader.load())
print(len(splitted_docs))

81


In [7]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings


'''
<https://huggingface.co/shibing624/text2vec-base-chinese/tree/main>
model_id: shibing624/text2vec-base-chinese
'''
# chinese_embedding_name = "/mnt/h/text2vec-base-chinese"
# chinese_embedding_name = "shibing624/text2vec-base-chinese"

# embeddings = HuggingFaceEmbeddings(
#     model_name=chinese_embedding_name,
#     model_kwargs={"device": "cuda"},
# )


embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    model_kwargs={"device": "cuda"},
)




c:\Users\benve\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
splitted_doc_embeddings = embedding_model.embed_documents([doc.page_content for doc in splitted_docs])

print(len(splitted_doc_embeddings), len(splitted_doc_embeddings[0]))

81 384


In [9]:
import chromadb
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings




class ChromadbEmbeddingFunction(EmbeddingFunction):
    def __init__(self, embedding_model: EmbeddingFunction):
        super().__init__()
        self.embedding_model = embedding_model
    def __call__(self, input: Documents) -> Embeddings:
        # embeddings = [self.embedding_model.embed_query(x) for x in input]
        embeddings = self.embedding_model.embed_documents(input)
        return embeddings



In [10]:

collection_name = 'llama2_demo'


# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
client = chromadb.Client()

exist_collection_name_list = [x.name for x in client.list_collections()]
# client.reset() 
for collection_name in exist_collection_name_list:
    client.delete_collection(collection_name)

collection = client.get_or_create_collection(collection_name, embedding_function=ChromadbEmbeddingFunction(embedding_model))

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    # embeddings=splitted_doc_embeddings, # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    documents=[doc.page_content for doc in splitted_docs], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    # metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on these!
    ids=[f"dd{i:05d}" for i in range(len(splitted_doc_embeddings))], # unique for each doc
    # ids=["f1", "f2"], # unique for each doc
)


# # Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["who is the woman?"],
    n_results=5,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)
# print(results)
distance_score = results['distances']
retrieved_documents = results['documents']
# print(distance_score)
# print(len(retrieved_documents[0]))
for doc in retrieved_documents[0]:
    print(doc)
    print()




her name, to elude the time police. No wonder she had never wanted her picture taken! And how
terrified she must have been on that long-ago day when she had stepped timidly into his office to
apply for a job! All alone in a strange generation, not knowing for sure whether her father's
concept of time was valid, not knowing for sure whether the man who would love her in his
-11-
forties would feel the same way toward her in his twenties. She had come back all right, just as

-8-
much as looked at another woman, and here in the space of less than a week he had not only
looked at one but had fallen in love with her.
Hope was dead in him when he climbed the hill on the fourth day—and then suddenly
alive again when he saw her standing in the sun. She was wearing a black dress this time, and he
should have guessed the reason for her absence; but he didn't—not till he came up to her and saw

"I'm Julie," she said. "Julie Danvers."
The name suited her. The same way the white dress suited her—t

In [112]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough


template = """Answer the question based only on the following context:

{context}

Question: {question}
"""



# template2 = """Answer the question based on the following context as well as 
#                 choosing the actions if you need more reasoning steps or information:

# {context}

# Action: ["Want to know more", "Answer found"]

# Question: {question}
# """
template2 = """Answer the question based on the following context as well as 
                choosing the actions, 
                

{context}

Action: ["Want to know more", "Respond that is certain"]

Question: {question}

repond in the following format:
    Action: 
    Respond:
    Potential Answer:
    Follow-up questions (>= 3):
"""

# prompt = ChatPromptTemplate.from_template(template)
prompt = ChatPromptTemplate.from_template(template2)
model = ChatOpenAI()


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


retriever_collected_docs = []
def retriever_func(question):
    global retriever_collected_docs
    results = collection.query(
        # query_texts=["who is the woman?"],
        query_texts=[question],
        n_results=5,
        # where={"metadata_field": "is_equal_to_this"}, # optional filter
        # where_document={"$contains":"search_string"}  # optional filter
    )
    # print(results)
    distance_score = results['distances']
    retrieved_documents = results['documents']
    
    distance_score = results['distances']
    retrieved_documents = results['documents']
    # print(distance_score)
    # print(len(retrieved_documents[0]))
    # retriever_collected_docs = []

    for doc in retrieved_documents[0]:
        retriever_collected_docs.append(doc)
        # print(doc)
        # print()
    return retrieved_documents
    

chain_asking_question = (
    # {"context": retriever | format_docs, "question": RunnablePassthrough()}
    {"context": retriever_func, "question": RunnablePassthrough()}
    | prompt
    | model
    # | StrOutputParser()
)

In [113]:
# chain_asking_question.invoke("What did the president say about technology?")
# chain_asking_question.invoke("What are main character's name? think it step by step")
# chain_asking_question.invoke("Who are main characters? think it step by step")
# chain_asking_question.invoke("What are characters' name? think it step by step")

# Direct asking
# chain_asking_question.invoke("Who does Randolph talk to?")
# chain_asking_question.invoke("Who does Mark talk to? Tell me her name.") # with instructions

# # Summarization
# chain_asking_question.invoke("What is the main idea of the article?")


# # Abstract idea / Emotional / Opinion
chain_asking_question.invoke("How does the man feel about the girl she meet on the hill top?")


# #  Indirect question / long-hopping question
# chain_asking_question.invoke("Who does Mark talk to?")
# chain_asking_question.invoke("What is the relationship between the girl Mark talked to on the hill and his wife?")
# chain_asking_question.invoke("Find out the relationship between the girl Mark talked to on the hill and his wife.")

# # frequency / Quantity
# chain_asking_question.invoke("What is the most frequent sentences in the article?")
# chain_asking_question.invoke("What is the representative sentences in the story talked by the girl?") 
# # the day before yesterday I saw a rabbit, and yesterday I saw a deer, and today, I saw you.

# # Ambiguous (Need Clarification)
# chain_asking_question.invoke("Who is Tom?") ## many Tom in different stories

AIMessage(content="Action: Respond that is certain\nRespond: The man is infatuated with the girl he met on the hilltop.\nPotential Answer: The man is deeply captivated by the girl he met on the hilltop. He finds her appearance and presence to be enchanting, and he feels a strong connection to her.\n\nFollow-up questions:\n1. What are some specific details that indicate the man is infatuated with the girl?\n2. How does the man's infatuation affect his daily life?\n3. Does the man have any doubts or reservations about his feelings for the girl?")

In [114]:
dandelion_girl_questions = []
with open("TheDandelionGirlQuestion.txt", "r") as f:
    line = f.readline()
    while line:
        dandelion_girl_questions.append(line)
        line = f.readline()


In [115]:
dandelion_girl_0_hop__question_chain_listxd  = []
dandelion_girl_2_hop__question_chain_listxd  = []

dandelion_girl_0_hop__gathered_docs_list2d = []
dandelion_girl_2_hop__gathered_docs_list2d = []

dandelion_girl_0_hop__response_list2d = []
dandelion_girl_2_hop__response_list2d = []

dandelion_girl_0_hop__potential_answer_listxd = []
dandelion_girl_2_hop__potential_answer_listxd = []

In [116]:
# print(retriever_collected_docs)


In [117]:
questions = dandelion_girl_questions[1:2]
for q in questions:

    response = chain_asking_question.invoke(q)

    dandelion_girl_0_hop__response_list2d.append(response)
    print(retriever_collected_docs)
    dandelion_girl_0_hop__gathered_docs_list2d.append(retriever_collected_docs)
    retriever_collected_docs = []
    
    resp_parsed = re.split(r"([\w \-\(\)>=3]+:)", response.content)

    assert (resp_parsed[1] == "Action:" and 
            resp_parsed[3] == "Respond:" and 
            resp_parsed[5] == "Potential Answer:" and 
            (resp_parsed[7] == "Follow-up questions:" or
             resp_parsed[7] == "Follow-up questions (>= 3):"
             )
        )
    
    action = resp_parsed[2].strip()

    
    action = resp_parsed[2].strip()
    respond = resp_parsed[4].strip()
    potential_answer = resp_parsed[6].strip()
    follow_up_questions = resp_parsed[8].strip().split("\n")

    dandelion_girl_0_hop__potential_answer_listxd.append(potential_answer)

    list_of_follow_up_questions = []
    for q in follow_up_questions:
        if q.strip() != "":
            list_of_follow_up_questions.append(q.strip())
    dandelion_girl_0_hop__question_chain_listxd.append(list_of_follow_up_questions)

    assert action in ["Want to know more", "Respond that is certain"], print(action)

['routine had caught up to him, and he had taken off into the woods without purpose or direction\nand finally he had come to the hill and had climbed it and seen the girl.\nHer eyes were blue, he saw when he came up to her—as blue as the sky that framed her\nslender silhouette. Her face was oval and young and soft and sweet. It evoked a déjà vu so\npoignant that he had to resist an impulse to reach out and touch her wind-kissed cheek; and even', 'that she loved him, that in a few short hours he would see her again. Surely even a run-down\ntime machine should have no trouble transporting her from the hamlet to the hill.\nHe arrived there early and sat down on the granite bench and waited for her to come out\nof the woods and climb the slope. He could feel the hammering of his heart and he knew that his\nhands were trembling. Day before yesterday I saw a rabbit, and yesterday a deer, and today,\nyou.', 'The next afternoon she did not show up either. Nor the next. He could neither eat nor

In [118]:
# respond
print(questions)
print(dandelion_girl_0_hop__potential_answer_listxd)
print(dandelion_girl_0_hop__gathered_docs_list2d)



['When was the story first published?\n']
['The story seems to involve a person who is reminiscing about someone named Anne and their past experiences together. The protagonist seems to have strong feelings for Anne and is haunted by memories of her.']
[['routine had caught up to him, and he had taken off into the woods without purpose or direction\nand finally he had come to the hill and had climbed it and seen the girl.\nHer eyes were blue, he saw when he came up to her—as blue as the sky that framed her\nslender silhouette. Her face was oval and young and soft and sweet. It evoked a déjà vu so\npoignant that he had to resist an impulse to reach out and touch her wind-kissed cheek; and even', 'that she loved him, that in a few short hours he would see her again. Surely even a run-down\ntime machine should have no trouble transporting her from the hamlet to the hill.\nHe arrived there early and sat down on the granite bench and waited for her to come out\nof the woods and climb the sl

In [119]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from operator import itemgetter


template_check_answer = """Check the following response to the question is correct or not:, 
    in the book "The Dandelion Girl", 
    the question is: {question}
    the response is: {response}

    Action: ["True", "False"]
    repond in the following format:
        Action: 
        Explaination:
"""

# prompt = ChatPromptTemplate.from_template(template)
prompt_check_answer = ChatPromptTemplate.from_template(template_check_answer)

chain_check_answer = (
    {"question": itemgetter("question"), "response": itemgetter("response")}
    | prompt_check_answer
    | model
    # | StrOutputParser()
)

In [120]:
checking_result_list = []
checking_TF_list = []
checking_explaination_list = []

In [121]:
for q, potential_answer in zip(questions, dandelion_girl_0_hop__potential_answer_listxd):
    response = chain_check_answer.invoke({"question":q, "response":potential_answer})

    resp_parsed = re.split(r"([\w \-\(\)>=3]+:)", response.content)
    assert (resp_parsed[1] == "Action:" and 
            resp_parsed[3] == "Explanation:" 
        )

    action = resp_parsed[2].strip()
    explanation = resp_parsed[4].strip()

    checking_result_list.append(response)
    checking_TF_list.append(action)
    checking_explaination_list.append(explanation)

In [122]:
print(questions)
print(dandelion_girl_0_hop__potential_answer_listxd)
print(dandelion_girl_0_hop__gathered_docs_list2d)
print(checking_TF_list)
print(checking_explaination_list)

for docs in dandelion_girl_0_hop__gathered_docs_list2d:
    print()
    print()
    for doc in docs:
        print(doc)

['When was the story first published?\n']
['The story seems to involve a person who is reminiscing about someone named Anne and their past experiences together. The protagonist seems to have strong feelings for Anne and is haunted by memories of her.']
[['routine had caught up to him, and he had taken off into the woods without purpose or direction\nand finally he had come to the hill and had climbed it and seen the girl.\nHer eyes were blue, he saw when he came up to her—as blue as the sky that framed her\nslender silhouette. Her face was oval and young and soft and sweet. It evoked a déjà vu so\npoignant that he had to resist an impulse to reach out and touch her wind-kissed cheek; and even', 'that she loved him, that in a few short hours he would see her again. Surely even a run-down\ntime machine should have no trouble transporting her from the hamlet to the hill.\nHe arrived there early and sat down on the granite bench and waited for her to come out\nof the woods and climb the sl

In [ ]:
multi_llm = OpenAI(n=4, best_of=4)
# embeddings = HypotheticalDocumentEmbedder.from_llm(
#     multi_llm, base_embeddings, "web_search"
# )
# result = embeddings.embed_query("Where is the Taj Mahal?")

# multi_response
# results = multi_llm.generate(["Where is the Taj Mahal?"])
# print(results)

# chain_tmp = (RunnablePassthrough() | multi_llm)
# print(chain_tmp.invoke("Where is the Taj Mahal?"))

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
import transformers
import torch
import warnings

from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.memory import ConversationBufferWindowMemory


# model_path = '/mnt/h/Chinese-Llama-2-7b-4bit'
model_path = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     load_in_4bit=True,
#     torch_dtype=torch.float16,
#     device_map='auto'
# )

pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

c:\Users\benve\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\benve\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


TypeError: Could not infer framework from class <class 'langchain.chat_models.openai.ChatOpenAI'>.

In [ ]:
# prompt_template = """<s>[INST] <<SYS>>
# {{ You are a helpful AI Assistant}}<<SYS>>
# ###

# Previous Conversation:
# '''
# {history}
# '''

# {{{input}}}[/INST]

# """
# prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])



# chain = ConversationChain(llm=llm, prompt=prompt)
# chain.run("What is the capital Of India?")

# memory = ConversationBufferWindowMemory(k=5)

# chain = ConversationChain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory
# )


In [ ]:

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer,\\
just say that you don't know, don't try to make up an answer. 

{context}

{history}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key='question', memory_key='history')

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(), llm=llm)
# retriever_from_llm = db.as_retriever()

In [ ]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever_from_llm)

In [ ]:
# retri_docs = compression_retriever.get_relevant_documents('主人公はだれですか？')
# print(retri_docs)

In [22]:
# import Pipe
# from statistics import mean

# @Pipe
# def running_average(iterable, width):
#     items = deque(maxlen=width)
#     for item in iterable:
#         items.append(item)
#         yield mean(items)

# list(range(20) | running_average(width=2))
# list(range(20) | running_average(width=10))


ModuleNotFoundError: No module named 'Pipe'

In [24]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from operator import itemgetter


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
model = ChatOpenAI()

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    # | model
)

In [26]:
print(chain.invoke({"foo": "hello", "bar": "world"}))

messages=[HumanMessage(content='what is 5 + 25')]
